In [1]:
!pip -q install rank_bm25
!pip -q install -U sentence-transformers
!pip -q install deep_translator
!pip -q install transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.5 MB/s eta 0:00:00


In [2]:
#Inference
import re
import torch
import numpy as np
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
#Set-up
#Corpus - download corpus
!gdown 1GrRGfYIaVm48opRgBF0L4RbS7sHwX2Jw #Vi-En ver.

#Function - data preprocessing
!gdown 1PgMXgc1ZJ-sqz-XR1bFu2cdR1q4kXu_P
from context_retrieval_vimedqa import *

In [45]:
def bm25_searcher(test, corpus, top_k=1):
  words = corpus['context_token'].tolist()
  bm25 = BM25Okapi(words)
  indexs = list(range(len(words)))
  contexts = []
  for question_index in range(len(test)):
    top_docs = []
    tokenized_query = test.iloc[question_index]['ques_token']
    ##### BM25 search (lexical search) #####
    top_indexs = bm25.get_top_n(tokenized_query, indexs, n=top_k)
    hits = [{'corpus_id' : index} for index in top_indexs]
    query = test.iloc[question_index]['question']
    ##### Re-Ranking #####
    cross_inp = [[query, corpus['topic'].iloc[hit['corpus_id']] + '. ' + corpus['context'].iloc[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:3]: #only top 3
      top_docs.append(corpus['eng_context'].iloc[hit['corpus_id']].replace("\n", " "))
    contexts.append('. '.join(top_docs))
  test['eng_context'] = contexts
  return test

def translate_df(df):
  df['question'] = df['question'].apply(translate_text)
  df['option_1'] = df['option_1'].apply(translate_text)
  df['option_2'] = df['option_2'].apply(translate_text)
  df['option_3'] = df['option_3'].apply(translate_text)
  df['option_4'] = df['option_4'].apply(translate_text)
  df['option_5'] = df['option_5'].apply(translate_text)
  df['option_6'] = df['option_6'].apply(translate_text)
  df['eng_context'] = df['eng_context']
  return df

In [4]:
#Model - download check point (trained on 2 epochs)
!gdown 14oj4IyRdGKhoeysIihFUU8IrsvFD8EwC

Downloading...
From: https://drive.google.com/uc?id=14oj4IyRdGKhoeysIihFUU8IrsvFD8EwC
To: /content/flan-t5-base-finetuned-MedMCQA_FrenchMCQA-v1.pt
100% 990M/990M [00:14<00:00, 68.6MB/s]


In [64]:
#Model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
#Load model
model.load_state_dict(torch.load("/content/flan-t5-base-finetuned-MedMCQA_FrenchMCQA-v1.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [71]:
def predict_per_sample(df, model, tokenizer):
  convert_to_binary = {'a': "10000", 'b':"01000", 'c':"00100", 'd':"00010", 'e':"00001",
                       'ab': "11000", 'ac':"10100", 'ad':"10010", 'ae':"10001",
                       'bc':"01100", 'bd':"01010", 'be':"01001", 'cd':"00110", 'ce':"00101", 'de':"00011",
                       'abc': "11100", 'abd':"11010", 'abe':"11001", 'acd':"10110", 'ace':"10101", 'ade':"10011",
                       'bcd': "01110", 'bce':"01101", 'bde':"01011", 'cde':"00111",
                       'abcd':"11110", 'abce':"11101", 'abde':"11011", 'acde':"10111", 'bcde':"01111",'abcde':"11111"}
  opt_no_lst = {2: 'AB', 3: 'ABC', 4: 'ABCD', 5: 'ABCDE', 6: 'ABCDEF', 7: 'ABCDEFG'}

  answers = []
  for idx, row in df.iterrows():
    context = row["eng_context"]
    question = row["question"]
    options = [re.sub(r"[A-E]\.", "", op).strip() for op in row[['option_1', 'option_2',
                                                                 'option_3', 'option_4',
                                                                 'option_5', 'option_6']].tolist() if str(op) != '']
    options_format = ['%s. %s' % (i, option) for i, option in zip(opt_no_lst[len(options)], options)]

    text_choices = ".\n".join(options_format)
    prompt = f"question: {question}. options: {text_choices}. context: {context}."

    model_inputs = tokenizer(prompt, truncation=True, return_tensors="pt")
    outputs = model.generate(**model_inputs, output_scores=True, max_new_tokens=30)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if prediction in convert_to_binary:
      answer = convert_to_binary[prediction]
    else:
      answer = '00000'
    answer = answer[:len(options)]
    answers.append(answer)
  return answers

def prediction(test_data, corpus_df, model, tokenizer, top_k=5):
  test_w_context_data = bm25_searcher(test_data, corpus_df, top_k=top_k)
  #test_w_context_data.to_csv("vi_context_public_test.csv", index = False)
  en_test_w_context_data = translate_df(test_w_context_data)
  #en_test_w_context_data.to_csv("en_context_public_test.csv")
  return predict_per_sample(en_test_w_context_data, model, tokenizer)

In [72]:
#Load corpus
corpus_path = '/content/ViMed_corpus_dict_envi.csv'
corpus_df = corpus_preprocessing(corpus_path)

In [73]:
#Preprocessing
test_df_path = "./public_test.csv"
test_data = test_set_preprocessing(test_df_path)

In [74]:
#Prediction
answers = prediction(test_data, corpus_df, model, tokenizer, top_k=5)

In [75]:
test_data['answer'] = answers
result_df = test_data[["id", "answer"]]
result_df.to_csv("answer_public_test_colab_v1-envi-3.csv")